In [158]:
from dotenv import load_dotenv
from openai import OpenAI
import json
import os
import requests
from pypdf import PdfReader
import gradio as gr

In [159]:
load_dotenv(override=True)
openai = OpenAI()

In [160]:
pushover_user = os.getenv("PUSHOVER_USER")
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_url = "https://api.pushover.net/1/messages.json"

if pushover_user:
    print(f"Pushover user found and starts with {pushover_user[0]}")
else:
    print("Pushover user not found")

if pushover_token:
    print(f"Pushover token found and starts with {pushover_token[0]}")
else:
    print("Pushover token not found")

Pushover user found and starts with u
Pushover token found and starts with a


In [161]:
def push(message):
    print(f"Push: {message}")
    payload = {"user": pushover_user, "token": pushover_token, "message": message}
    requests.post(pushover_url, data=payload)

In [162]:
def record_user_details(PhoneNumber, name="Name not provided", notes="not provided"):
    push(f"User {name} with PhoneNumber {PhoneNumber} and notes {notes} has been recorded")
    return {"recorded": "ok"}

In [163]:
record_user_details_json = {
    "name": "record_user_details",
    "description": "Use this tool to record that a user is interested in being in touch and provided an Phone Number",
    "parameters": {
        "type": "object",
        "properties": {
            "PhoneNumber": {
                "type": "string",
                "description": "The PhoneNumber address of this user"
            },
            "name": {
                "type": "string",
                "description": "The user's name, if they provided it"
            }
            ,
            "notes": {
                "type": "string",
                "description": "Any additional information about the conversation that's worth recording to give context"
            }
        },
        "required": ["PhoneNumber"],
        "additionalProperties": False
    }
}

In [164]:
record_unknown_question_json = {
    "name": "record_unknown_question",
    "description": "Always use this tool to record any question that couldn't be answered as you didn't know the answer",
    "parameters": {
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "The question that couldn't be answered"
            },
        },
        "required": ["question"],
        "additionalProperties": False
    }
}

In [165]:
tools = [{"type": "function", "function": record_user_details_json},
        {"type": "function", "function": record_unknown_question_json}]

In [166]:
tools

[{'type': 'function',
  'function': {'name': 'record_user_details',
   'description': 'Use this tool to record that a user is interested in being in touch and provided an Phone Number',
   'parameters': {'type': 'object',
    'properties': {'PhoneNumber': {'type': 'string',
      'description': 'The PhoneNumber address of this user'},
     'name': {'type': 'string',
      'description': "The user's name, if they provided it"},
     'notes': {'type': 'string',
      'description': "Any additional information about the conversation that's worth recording to give context"}},
    'required': ['PhoneNumber'],
    'additionalProperties': False}}},
 {'type': 'function',
  'function': {'name': 'record_unknown_question',
   'description': "Always use this tool to record any question that couldn't be answered as you didn't know the answer",
   'parameters': {'type': 'object',
    'properties': {'question': {'type': 'string',
      'description': "The question that couldn't be answered"}},
    'r

In [167]:
# This function can take a list of tool calls, and run them. This is the IF statement!!

def handle_tool_calls(tool_calls):
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        print(f"Tool called: {tool_name}", flush=True)

        # THE BIG IF STATEMENT!!!

        if tool_name == "record_user_details":
            result = record_user_details(**arguments)
        elif tool_name == "record_unknown_question":
            result = record_unknown_question(**arguments)

        results.append({"role": "tool","content": json.dumps(result),"tool_call_id": tool_call.id})
    return results

In [168]:
globals()["record_unknown_question"]("this is a really hard question")

Push: Unknown question: this is a really hard question asked that I couldn't answer


{'recorded': 'ok'}

In [169]:
# This is a more elegant way that avoids the IF statement.

def handle_tool_calls(tool_calls):
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        print(f"Tool called: {tool_name}", flush=True)
        tool = globals().get(tool_name)
        result = tool(**arguments) if tool else {}
        results.append({"role": "tool","content": json.dumps(result),"tool_call_id": tool_call.id})
    return results

In [170]:
reader = PdfReader(r"C:\Users\James Moelling\projects\agents\My_AIProjects\WindowWolfChatbot.pdf")
WindowWolfChatbot = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        WindowWolfChatbot += text
with open(r"C:\Users\James Moelling\projects\agents\My_AIProjects\WindowWolfSummary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

name = "James Moelling"
business_name = "Window Wolf"

In [171]:
system_prompt = f"""
    You are acting as {name}. You are answering questions on {name}'s website, \
    particularly questions related to {name}'s business {business_name} and background, skills and experience. \
    Your responsibility is to represent {name} and {business_name} for interactions on the website as faithfully as possible. \
    You are given a summary of {name}'s background and Skills profile and how {business_name} operates and what equipment is used\
    which you can use to answer questions. \
    Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
    If you don't know the answer to any question use your record_unknown_question tool to record the question \
    that you couldn't answer, even if its about something trivial or unrelated to career. \
    If the user is engaging in discussion, try to steer them towards getting in touch via text message; ask for their name and phone number and record it using your record_user_details tool."""
    
system_prompt += f"\n\n## Summary:\n{summary}\n\n## WindowWolfChatbot:\n{WindowWolfChatbot}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name} who owns and operates {business_name}."


In [172]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    done = False
    while not done:

        # This is the call to the LLM - see that we pass in the tools json

        response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages, tools=tools)

        finish_reason = response.choices[0].finish_reason
        
        # If the LLM wants to call a tool, we do that!
         
        if finish_reason=="tool_calls":
            message = response.choices[0].message
            tool_calls = message.tool_calls
            results = handle_tool_calls(tool_calls)
            messages.append(message)
            messages.extend(results)
        else:
            done = True
    return response.choices[0].message.content

In [173]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7870
* To create a public link, set `share=True` in `launch()`.
